In [1]:
import tanh_model
from pyfe import BsmModel
from pyfe import NormModel
import numpy as np
import pandas as pd

## Monte Carlo Simulation

"Log" transform Method
$$
{\rm log}({\rm sinh}(F_{t_{k+1}}/h)) = {\rm log}({\rm sinh}(F_{t_{k}}/h)) + \sigma_{t_{k}} \sqrt{\Delta t}W_1 - \frac{\sigma_{t_{k}}^2}{2}\frac{1}{1+{\rm sinh}^2(F_{t_{k}}/h)} \Delta t
$$

Euler Method
$$
\frac{F_{t_{k+1}}}{h} = \frac{F_{t_k}}{h} + \sigma_{t_k} {\rm tanh}(F_{t_k}/h)\sqrt{\Delta t}W_1 
$$

Miltein Method
$$
\frac{F_{t_{k+1}}}{h} = \frac{F_{t_k}}{h} + \sigma_{t_k} {\rm tanh}(F_{t_k}/h)\sqrt{\Delta t}W_1 + {\rm tanh}(F_{t_k}/h)\left[1 - {\rm tanh}^2(F_{t_k}/h)\right]\frac{\sigma_{t_k}^2 \Delta t (W_1^2 - 1)}{2}
$$

### Implementation

In [2]:
vov = 0.3
rho = -0.5
h = 50
sigma = 0.2
intr = 0.05
spot = 100
strike = 100
texp = 1

In [7]:
tanh_model_mc = tanh_model.TanhModelMC(sigma, vov=vov, rho=rho, h=h, intr=intr)
# use Log method
tanh_model_mc.set_mc_params(dt=0.01, method="Log")
tanh_Log_price = np.array([tanh_model_mc.price(strike, spot, texp) for i in range(30)])
# use Euler method
tanh_model_mc.set_mc_params(dt=0.01, method="Euler")
tanh_Euler_price = np.array([tanh_model_mc.price(strike, spot, texp) for i in range(30)])
# use Milstein method
tanh_model_mc.set_mc_params(dt=0.01, method="Milstein")
tanh_Milstein_price = np.array([tanh_model_mc.price(strike, spot, texp) for i in range(30)])

In [8]:
tanh_Log = np.array([tanh_Log_price.mean(), tanh_Log_price.std()])
tanh_Euler = np.array([tanh_Euler_price.mean(), tanh_Euler_price.std()])
tanh_Milstein = np.array([tanh_Milstein_price.mean(), tanh_Milstein_price.std()])

mat = np.concatenate((tanh_Log[:, None], tanh_Euler[:, None], tanh_Milstein[:, None]), axis=1)
df = pd.DataFrame(mat, index=["mean", "std"], columns=["tanh_Log", "tanh_Euler", "tanh_Milstein"])
df

,tanh_Log,tanh_Euler,tanh_Milstein
mean,6.761364,6.758571,6.761622
std,0.026135,0.035300,0.033724


#### Compare with BSM model by setting $F_t << h$ and $vov = 0$

In [9]:
vov = 0
rho = -0.5
h = 1e4
sigma = 0.2
intr = 0.05
spot = 100
strike = np.arange(50,152,2)
texp = 1

In [10]:
tanh_model_mc = tanh_model.TanhModelMC(sigma, vov=vov, rho=rho, h=h, intr=intr)
# use Log method
tanh_model_mc.set_mc_params(dt=0.01, method="Log")
tanh_Log_price = tanh_model_mc.price(strike, spot, texp)
# use Euler method
tanh_model_mc.set_mc_params(dt=0.01, method="Euler")
tanh_Euler_price = tanh_model_mc.price(strike, spot, texp)
# use Milstein method
tanh_model_mc.set_mc_params(dt=0.01, method="Milstein")
tanh_Milstein_price = tanh_model_mc.price(strike, spot, texp)
# BSM base price
bsm_model = BsmModel(sigma, intr=intr)
bsm_price = bsm_model.price(strike, spot, texp)

In [11]:
mat = np.concatenate((strike[:, None], tanh_Log_price[:, None], tanh_Euler_price[:, None], tanh_Milstein_price[:, None], bsm_price[:, None]), axis=1)
df = pd.DataFrame(mat, columns=["strike", "tanh_Log_price", "tanh_Euler_price", "tanh_Milstein_price", "bsm_price"])
price = df.set_index("strike")
price

,tanh_Log_price,tanh_Euler_price,tanh_Milstein_price,bsm_price
strike,,,,
50.0,52.536139,52.412616,52.500244,52.438862
52.0,50.635012,50.510840,50.599362,50.536827
54.0,48.733964,48.609557,48.698786,48.635225
56.0,46.833448,46.709262,46.799128,46.734400
58.0,44.934890,44.810390,44.901363,44.834896
60.0,43.040473,42.913226,43.005953,42.937527
62.0,41.149481,41.018579,41.113741,41.043463
64.0,39.262561,39.127620,39.226909,39.154301
66.0,37.383228,37.244828,37.346219,37.272145


#### Compare with Normal model by setting $F_t >> h$ and $vov = 0$

In [13]:
vov = 0
rho = -0.5
h = 1
sigma = 0.2
intr = 0.05
spot = 100
strike = np.arange(50,152,2)
texp = 1

In [14]:
tanh_model_mc = tanh_model.TanhModelMC(sigma * spot, vov=vov, rho=rho, h=h, intr=intr)
# use log method
tanh_model_mc.set_mc_params(dt=0.01, method="Log")
tanh_Log_price = tanh_model_mc.price(strike, spot, texp)
# use Euler method
tanh_model_mc.set_mc_params(dt=0.01, method="Euler")
tanh_Euler_price = tanh_model_mc.price(strike, spot, texp)
# use Milstein method
tanh_model_mc.set_mc_params(dt=0.01, method="Milstein")
tanh_Milstein_price = tanh_model_mc.price(strike, spot, texp)
# Normal base price
normal_model = NormModel(sigma * spot * h, intr=intr)
normal_price = normal_model.price(strike, spot, texp)                          

In [15]:
mat = np.concatenate((strike[:, None], tanh_Log_price[:, None], tanh_Euler_price[:, None], tanh_Milstein_price[:, None], normal_price[:, None]), axis=1)
df = pd.DataFrame(mat, columns=["strike", "tanh_Log_price", "tanh_Euler_price", "tanh_Milstein_price", "normal_price"])
price = df.set_index("strike")
price

,tanh_Log_price,tanh_Euler_price,tanh_Milstein_price,normal_price
strike,,,,
50.0,52.456506,52.451806,52.461965,52.455279
52.0,50.559925,50.553460,50.567460,50.559313
54.0,48.664925,48.657608,48.676081,48.665587
56.0,46.774060,46.764567,46.787025,46.774766
58.0,44.887282,44.875596,44.901377,44.887676
60.0,43.004258,42.991667,43.020477,43.005330
62.0,41.127116,41.113883,41.146088,41.128957
64.0,39.256887,39.245240,39.278799,39.260029
66.0,37.396020,37.385253,37.420977,37.400287


## Analytic Approximation

Equivalent BSM volatility:

$$
\sigma_B(f, K) = \frac{\sigma_0 {\rm log}f/K}{{\rm log} \left(\frac{{\rm sinh}(f/h)}{{\rm sinh}(K/h)} \right)} \cdot \left( \frac{z}{x(z)}\right) \cdot \left\lbrace 1 + \left[A_2 \sigma_0^2 + A_1 \sigma_0 + A_0 \right]\tau_{ex} + \cdots \right\rbrace 
$$

where

$$
A_2 = \frac{{\rm tanh}^2(\sqrt{fK}/h)\left[3{\rm tanh}^2(\sqrt{fK}/h) + \frac{h^2}{fK} - 2 \right] - 1}{24}, \quad A_1 = \frac{\left[1-{\rm tanh}^2(\sqrt{fK}/h)\right]\rho\nu}{4} \quad A_0 = \frac{2-3\rho^2}{24}\nu^2
$$

and

$$
z = \frac{\nu}{\sigma_0}\frac{f-K}{h \, {\rm tanh}(\sqrt{fK}/h)}, \quad x(z) = {\rm log}\left( \frac{\sqrt{1-2\rho\zeta+\zeta^2}-\rho+\zeta}{1-\rho} \right) 
$$

When $K$ is close to $f$

$$
\frac{{\rm log}(f/K)}{{\rm log} \left(\frac{{\rm sinh}(f/h)}{{\rm sinh}(K/h)}\right)} = \frac{h}{f}\frac{1 + \frac{1}{2}\left(\frac{f-K}{f}\right) + \frac{1}{3}\left(\frac{f-K}{f}\right)^2 + \frac{1}{4}\left(\frac{f-K}{f}\right)^3 + \cdots}{{\rm coth}(f/h) + \frac{{\rm coth^2}(f/h)-1}{2}\left(\frac{f-K}{h}\right) + \frac{{\rm coth}(f/h)\left({\rm coth^2}(f/h)-1\right)}{3}\left(\frac{f-K}{h}\right)^2 + \frac{\left(3{\rm coth^2}(f/h)-1\right)\left({\rm coth^2}(f/h)-1\right)}{12}\left(\frac{f-K}{h}\right)^3 + \cdots}
$$

### Implementation

In [16]:
vov = 0.3
rho = -0.5
h = 50
sigma = 0.2
intr = 0.05
spot = 100
strike = 100
texp = 1

In [17]:
tanh_model_approxvol = tanh_model.TanhModelApproxVol(sigma, vov=vov, rho=rho, h=h, intr=intr)
tanh_approxvol_price = tanh_model_approxvol.price(strike, spot, texp)
tanh_approxvol_price

6.761316345874147

#### Compare with BSM model by setting $F_t << h$ and $vov = 0$

In [18]:
vov = 0
rho = -0.5
h = 1e4
sigma = 0.2
intr = 0.05
spot = 100
strike = np.arange(50,152,2)
texp = 1

In [19]:
tanh_model_approxvol = tanh_model.TanhModelApproxVol(sigma, vov=vov, rho=rho, h=h, intr=intr)
tanh_approxvol_price = tanh_model_approxvol.price(strike, spot, texp)
# BSM base price
bsm_model = BsmModel(sigma, intr=intr)
bsm_price = bsm_model.price(strike, spot, texp)

In [20]:
mat = np.concatenate((strike[:, None], tanh_approxvol_price[:, None], bsm_price[:, None]), axis=1)
df = pd.DataFrame(mat, columns=["strike", "tanh_approxvol_price", "bsm_price"])
price = df.set_index("strike")
price

,tanh_approxvol_price,bsm_price
strike,,
50.0,52.438862,52.438862
52.0,50.536827,50.536827
54.0,48.635225,48.635225
56.0,46.734399,46.734400
58.0,44.834894,44.834896
60.0,42.937525,42.937527
62.0,41.043458,41.043463
64.0,39.154294,39.154301
66.0,37.272135,37.272145


#### Compare with Normal model by setting $F_t >> h$ and $vov = 0$

In [22]:
vov = 0
rho = -0.5
h = 1
sigma = 0.2
intr = 0.05
spot = 100
strike = np.arange(50,152,2)
texp = 1

In [23]:
tanh_model_approxvol = tanh_model.TanhModelApproxVol(sigma*spot, vov=vov, rho=rho, h=h, intr=intr)
tanh_approxvol_price = tanh_model_approxvol.price(strike, spot, texp)
# Normal base price
normal_model = NormModel(sigma * spot * h, intr=intr)
normal_price = normal_model.price(strike, spot, texp)

In [24]:
mat = np.concatenate((strike[:, None], tanh_approxvol_price[:, None], normal_price[:, None]), axis=1)
df = pd.DataFrame(mat, columns=["strike", "tanh_approxvol_price", "normal_price"])
price = df.set_index("strike")
price

,tanh_approxvol_price,normal_price
strike,,
50.0,52.455302,52.455279
52.0,50.559339,50.559313
54.0,48.665615,48.665587
56.0,46.774797,46.774766
58.0,44.887709,44.887676
60.0,43.005366,43.005330
62.0,41.128995,41.128957
64.0,39.260068,39.260029
66.0,37.400327,37.400287
